# Fetch 10 Years OHLCV Data for Insurance HOSE Stocks

This notebook fetches 10 years of historical OHLCV (Open, High, Low, Close, Volume) data for all insurance stocks listed on HOSE and saves it to a CSV file for model training.


In [10]:
# Import required libraries
import pandas as pd
import numpy as np
from vnstock import Vnstock
from datetime import datetime, timedelta
import time
import os

# Set up paths
DATA_RAW_PATH = '../data/raw'
DATA_PROCESSED_PATH = '../data/processed'

# Create processed directory if it doesn't exist
os.makedirs(DATA_PROCESSED_PATH, exist_ok=True)

print("Libraries imported successfully!")
print(f"Data will be saved to: {DATA_PROCESSED_PATH}")


Libraries imported successfully!
Data will be saved to: ../data/processed


In [11]:
# Load insurance HOSE stock list
insurance_stocks_df = pd.read_csv(f'{DATA_RAW_PATH}/insurance_hose_ticket_name.csv')
insurance_stocks_list = insurance_stocks_df['Mã CK'].tolist()

print(f"Found {len(insurance_stocks_list)} insurance stocks:")
print(insurance_stocks_list)
print("\nStock details:")
print(insurance_stocks_df)


Found 5 insurance stocks:
['BIC', 'BMI', 'BVH', 'MIG', 'PGI']

Stock details:
    Sàn           Ngành Mã CK  \
0  HOSE  Ngành Bảo hiểm   BIC   
1  HOSE  Ngành Bảo hiểm   BMI   
2  HOSE  Ngành Bảo hiểm   BVH   
3  HOSE  Ngành Bảo hiểm   MIG   
4  HOSE  Ngành Bảo hiểm   PGI   

                                         Tên công ty  
0  Tổng CTCP Bảo hiểm Ngân hàng Đầu tư và Phát tr...  
1                                 Tổng CTCP Bảo Minh  
2                                  Tập đoàn Bảo Việt  
3                        Tổng CTCP Bảo hiểm Quân Đội  
4                      Tổng CTCP Bảo hiểm Petrolimex  


In [12]:
# Configure date range for 10 years of data
# Fetch data from 2014-01-01 to 2024-12-31 (10 years)
START_DATE = '2014-01-01'
END_DATE = '2024-12-31'
INTERVAL = '1D'  # Daily data

print(f"Date range: {START_DATE} to {END_DATE}")
print(f"Interval: {INTERVAL}")


Date range: 2014-01-01 to 2024-12-31
Interval: 1D


In [13]:
# Initialize Vnstock client
vnstock_client = Vnstock()

# Test with one stock first to understand data structure
test_symbol = insurance_stocks_list[0]
print(f"Testing with stock: {test_symbol}")

try:
    stock = vnstock_client.stock(symbol=test_symbol, source='VCI')
    test_df = stock.quote.history(start=START_DATE, end=END_DATE, interval=INTERVAL)
    
    print(f"\nSuccessfully fetched data for {test_symbol}")
    print(f"Shape: {test_df.shape}")
    print(f"\nColumns: {test_df.columns.tolist()}")
    print(f"\nFirst few rows:")
    print(test_df.head())
    print(f"\nData types:")
    print(test_df.dtypes)
    print(f"\nDate range in data: {test_df.index.min()} to {test_df.index.max()}")
except Exception as e:
    print(f"Error fetching test data: {e}")


Testing with stock: BIC

Successfully fetched data for BIC
Shape: (2872, 6)

Columns: ['time', 'open', 'high', 'low', 'close', 'volume']

First few rows:
        time  open  high   low  close  volume
0 2013-07-08  3.30  3.41  3.30   3.37    4960
1 2013-07-09  3.30  3.34  3.27   3.34    6400
2 2013-07-10  3.34  3.34  3.24   3.34    5580
3 2013-07-11  3.34  3.34  3.34   3.34     110
4 2013-07-12  3.41  3.41  3.27   3.37   21170

Data types:
time      datetime64[ns]
open             float64
high             float64
low              float64
close            float64
volume             int64
dtype: object

Date range in data: 0 to 2871


In [14]:
# Function to fetch OHLCV data for a single stock
def fetch_stock_data(symbol, start_date, end_date, interval='1D', source='VCI', retry_count=3):
    """
    Fetch historical OHLCV data for a single stock.
    
    Parameters:
    -----------
    symbol : str
        Stock symbol (e.g., 'BIC', 'BMI')
    start_date : str
        Start date in 'YYYY-MM-DD' format
    end_date : str
        End date in 'YYYY-MM-DD' format
    interval : str
        Data interval (default: '1D' for daily)
    source : str
        Data source (default: 'VCI')
    retry_count : int
        Number of retry attempts if request fails
        
    Returns:
    --------
    pd.DataFrame or None
        DataFrame with OHLCV data, or None if failed
    """
    for attempt in range(retry_count):
        try:
            stock = vnstock_client.stock(symbol=symbol, source=source)
            df = stock.quote.history(start=start_date, end=end_date, interval=interval)
            
            if df is not None and not df.empty:
                # Add symbol column to identify the stock
                df['symbol'] = symbol
                # Rename 'time' column to 'date' if it exists
                if 'time' in df.columns:
                    df = df.rename(columns={'time': 'date'})
                return df
            else:
                print(f"  Warning: Empty data returned for {symbol}")
                return None
                
        except Exception as e:
            if attempt < retry_count - 1:
                print(f"  Attempt {attempt + 1} failed for {symbol}: {e}. Retrying...")
                time.sleep(2)  # Wait 2 seconds before retry
            else:
                print(f"  Error fetching {symbol} after {retry_count} attempts: {e}")
                return None
    
    return None

print("Function defined successfully!")


Function defined successfully!


In [17]:
# Fetch data for all insurance stocks
all_data_frames = []
failed_stocks = []

print(f"Starting to fetch data for {len(insurance_stocks_list)} stocks...")
print("=" * 60)

for idx, symbol in enumerate(insurance_stocks_list, 1):
    print(f"[{idx}/{len(insurance_stocks_list)}] Fetching data for {symbol}...", end=' ')
    
    df = fetch_stock_data(symbol, START_DATE, END_DATE, INTERVAL)
    
    if df is not None and not df.empty:
        all_data_frames.append(df)
        print(f"✓ Success ({len(df)} rows)")
    else:
        failed_stocks.append(symbol)
        print(f"✗ Failed")
    
    # Add a small delay to avoid rate limiting
    time.sleep(1)

print("=" * 60)
print(f"\nSummary:")
print(f"  Successfully fetched: {len(all_data_frames)} stocks")
print(f"  Failed: {len(failed_stocks)} stocks")
if failed_stocks:
    print(f"  Failed stocks: {failed_stocks}")


Starting to fetch data for 5 stocks...
[1/5] Fetching data for BIC... ✓ Success (2872 rows)
[2/5] Fetching data for BMI... ✓ Success (2872 rows)
[3/5] Fetching data for BVH... ✓ Success (2872 rows)
[4/5] Fetching data for MIG... ✓ Success (1914 rows)
[5/5] Fetching data for PGI... ✓ Success (2872 rows)

Summary:
  Successfully fetched: 5 stocks
  Failed: 0 stocks


In [18]:
# Combine all dataframes into one
if all_data_frames:
    combined_df = pd.concat(all_data_frames, ignore_index=True)
    
    # Rename 'time' to 'date' if it exists (for consistency)
    if 'time' in combined_df.columns and 'date' not in combined_df.columns:
        combined_df = combined_df.rename(columns={'time': 'date'})
    
    print(f"Combined dataset shape: {combined_df.shape}")
    print(f"\nColumns: {combined_df.columns.tolist()}")
    print(f"\nFirst few rows:")
    print(combined_df.head(10))
    print(f"\nLast few rows:")
    print(combined_df.tail(10))
    print(f"\nUnique stocks: {combined_df['symbol'].unique()}")
    print(f"\nDate range: {combined_df['date'].min()} to {combined_df['date'].max()}")
    print(f"\nData info:")
    print(combined_df.info())
else:
    print("No data to combine!")


Combined dataset shape: (13402, 7)

Columns: ['date', 'open', 'high', 'low', 'close', 'volume', 'symbol']

First few rows:
        date  open  high   low  close  volume symbol
0 2013-07-08  3.30  3.41  3.30   3.37    4960    BIC
1 2013-07-09  3.30  3.34  3.27   3.34    6400    BIC
2 2013-07-10  3.34  3.34  3.24   3.34    5580    BIC
3 2013-07-11  3.34  3.34  3.34   3.34     110    BIC
4 2013-07-12  3.41  3.41  3.27   3.37   21170    BIC
5 2013-07-15  3.41  3.41  3.37   3.41    6510    BIC
6 2013-07-16  3.41  3.44  3.41   3.44    6000    BIC
7 2013-07-17  3.44  3.44  3.20   3.37   10280    BIC
8 2013-07-18  3.34  3.37  3.27   3.34    2920    BIC
9 2013-07-19  3.41  3.41  3.30   3.34    4740    BIC

Last few rows:
            date   open   high    low  close  volume symbol
13392 2024-12-18  21.71  21.71  21.71  21.71    3953    PGI
13393 2024-12-19  21.71  21.71  21.71  21.71       0    PGI
13394 2024-12-20  21.71  21.71  21.71  21.71       0    PGI
13395 2024-12-23  21.71  21.71  21.71 

In [19]:
# Check for missing values and data quality
if all_data_frames:
    print("Data Quality Check:")
    print("=" * 60)
    print("\nMissing values per column:")
    print(combined_df.isnull().sum())
    print("\nMissing values percentage:")
    print((combined_df.isnull().sum() / len(combined_df) * 100).round(2))
    print("\nBasic statistics:")
    print(combined_df.describe())
    
    # Check rows per stock
    print("\nRows per stock:")
    print(combined_df['symbol'].value_counts().sort_index())


Data Quality Check:

Missing values per column:
date      0
open      0
high      0
low       0
close     0
volume    0
symbol    0
dtype: int64

Missing values percentage:
date      0.0
open      0.0
high      0.0
low       0.0
close     0.0
volume    0.0
symbol    0.0
dtype: float64

Basic statistics:
                                date          open          high  \
count                          13402  13402.000000  13402.000000   
mean   2019-07-15 12:32:39.826891520     19.686685     19.976933   
min              2013-07-08 00:00:00      2.890000      2.990000   
25%              2016-11-14 00:00:00      9.390000      9.530000   
50%              2019-08-23 00:00:00     13.500000     13.710000   
75%              2022-04-29 00:00:00     23.430000     23.610000   
max              2024-12-31 00:00:00     90.880000     92.210000   
std                              NaN     16.589533     16.835326   

                low         close        volume  
count  13402.000000  13402.00000

In [20]:
# Save combined data to CSV
if all_data_frames:
    output_filename = f'{DATA_PROCESSED_PATH}/insurance_hose_ohlcv_10years.csv'
    
    # Ensure date column is properly formatted
    # Rename 'time' to 'date' if it still exists
    if 'time' in combined_df.columns and 'date' not in combined_df.columns:
        combined_df = combined_df.rename(columns={'time': 'date'})
    
    if 'date' in combined_df.columns:
        combined_df['date'] = pd.to_datetime(combined_df['date'])
        combined_df = combined_df.sort_values(['symbol', 'date'])
    
    # Save to CSV
    combined_df.to_csv(output_filename, index=False)
    
    print(f"✓ Data saved successfully to: {output_filename}")
    print(f"  Total rows: {len(combined_df):,}")
    print(f"  Total columns: {len(combined_df.columns)}")
    print(f"  File size: {os.path.getsize(output_filename) / (1024*1024):.2f} MB")
    
    # Verify the saved file
    print("\nVerifying saved file...")
    verify_df = pd.read_csv(output_filename)
    print(f"  Verified: {len(verify_df)} rows loaded from CSV")
    print(f"  Columns: {verify_df.columns.tolist()}")
else:
    print("No data to save!")


✓ Data saved successfully to: ../data/processed/insurance_hose_ohlcv_10years.csv
  Total rows: 13,402
  Total columns: 7
  File size: 0.56 MB

Verifying saved file...
  Verified: 13402 rows loaded from CSV
  Columns: ['date', 'open', 'high', 'low', 'close', 'volume', 'symbol']


## Summary

The OHLCV data has been successfully fetched and saved. The dataset is ready for:
- Feature engineering (calculating technical indicators)
- Linear regression model training
- Time series analysis

**Next steps:**
1. Load the CSV file: `pd.read_csv('../data/processed/insurance_hose_ohlcv_10years.csv')`
2. Perform feature engineering (calculate 20 technical indicators)
3. Prepare data for linear regression model training
